In [2]:
import sqlite3

In [3]:
conn = sqlite3.connect("Database/news_data.db")

In [ ]:
def create_new_news_table(Ticker, conn):
    """ 
    Ticker: str, the name of the table to be created.
    """
    cursor = conn.cursor()
    cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {Ticker} (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            Ticker TEXT NOT NULL,
            title TEXT NOT NULL,
            summary TEXT NOT NULL,
            url TEXT NOT NULL,
            full_article TEXT,
            source TEXT NOT NULL,
            publish_timestamp TEXT NOT NULL,
            day date,
            hour time,
            minute time,
            category_within_source TEXT NOT NULL,
            overall_sentiment_score TEXT NOT NULL,
            overall_sentiment_label TEXT NOT NULL,
            sector TEXT,
            Trading_price_at_release float ,
            Trading_price_1h_after_release float ,
            Trading_price_change float
        )
    ''')
    conn.commit()

def create_new_price_table(Ticker, conn):
    """ 
    Ticker: str, the name of the table to be created.
    """
    cursor = conn.cursor()
    cursor.execute(f'''
    CREATE TABLE IF NOT EXISTS {Ticker}PriceHistory (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        date TEXT NOT NULL,
        open REAL,
        high REAL,
        low REAL,
        close REAL,
        volume INTEGER,
        ticker TEXT NOT NULL,
        UNIQUE(ticker, date)
    )
    ''')
    conn.commit()

def close_connection(conn):
    conn.close()

def insert_data(Ticker, data, conn):
    """
    Ticker: str, the name of the table to insert data into.
    data: list of tuples, each tuple contains the data to be inserted.
    """
    cursor = conn.cursor()



In [29]:
create_new_price_table("AAPL", conn)

In [15]:
import requests
import pandas as pd
import datetime

# Replace with your Tiingo API token
API_TOKEN = '619a6ea449a7a5e044a79c9a2293eee49c339034'
SYMBOL = 'AAPL'

# Dates: 2 years back from today
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=730)

# Tiingo supports up to 1 year per request for intraday, so split in chunks
def fetch_tiingo_intraday(symbol, start, end):
    url = f"https://api.tiingo.com/iex/{symbol}/prices"
    headers = {
        'Content-Type': 'application/json'
    }
    params = {
        'startDate': start.isoformat(),
        'endDate': end.isoformat(),
        'resampleFreq': '1hour',  # 1-hour intervals
        'columns': 'open,high,low,close,volume',
        'token': API_TOKEN
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.text}")
        return []

    return response.json()

# Split into two chunks (because Tiingo limits range)
middle_date = start_date + datetime.timedelta(days=365)

print(f"Fetching {SYMBOL} hourly data from {start_date} to {middle_date}...")
data1 = fetch_tiingo_intraday(SYMBOL, start_date, middle_date)

print(f"Fetching {SYMBOL} hourly data from {middle_date + datetime.timedelta(days=1)} to {end_date}...")
data2 = fetch_tiingo_intraday(SYMBOL, middle_date + datetime.timedelta(days=1), end_date)

# Combine & convert to DataFrame
all_data = data1 + data2
df = pd.DataFrame(all_data)

# Convert timestamps to datetime
df['date'] = pd.to_datetime(df['date'])

# Show sample
print(df[['date', 'open', 'high', 'low', 'close', 'volume']].head())


Fetching AAPL hourly data from 2023-04-24 to 2024-04-23...
Fetching AAPL hourly data from 2024-04-24 to 2025-04-23...
                       date     open     high      low    close   volume
0 2023-04-24 14:00:00+00:00  165.175  165.425  164.020  164.035  71532.0
1 2023-04-24 15:00:00+00:00  164.040  164.410  163.900  164.140  57530.0
2 2023-04-24 16:00:00+00:00  164.140  164.620  164.040  164.520  40919.0
3 2023-04-24 17:00:00+00:00  164.515  165.215  164.385  165.135  41630.0
4 2023-04-24 18:00:00+00:00  165.140  165.440  164.980  165.295  55066.0


In [19]:
df.head(20)

,date,open,high,low,close,volume
0,2023-04-24 14:00:00+00:00,165.175,165.425,164.020,164.035,71532.0
1,2023-04-24 15:00:00+00:00,164.040,164.410,163.900,164.140,57530.0
2,2023-04-24 16:00:00+00:00,164.140,164.620,164.040,164.520,40919.0
3,2023-04-24 17:00:00+00:00,164.515,165.215,164.385,165.135,41630.0
4,2023-04-24 18:00:00+00:00,165.140,165.440,164.980,165.295,55066.0
5,2023-04-24 19:00:00+00:00,165.300,165.590,165.085,165.355,127022.0
6,2023-04-25 14:00:00+00:00,166.040,166.270,165.060,165.225,136729.0
7,2023-04-25 15:00:00+00:00,165.220,165.650,164.790,164.870,100585.0
8,2023-04-25 16:00:00+00:00,164.870,164.925,164.385,164.620,71567.0
9,2023-04-25 17:00:00+00:00,164.620,164.820,163.800,163.930,111561.0


In [ ]:
import requests
import pandas as pd
import datetime
import time

def fetch_10min_tiingo_data(ticker, api_token):
    """
    Fetches 2 years of 10-minute interval historical data for a given stock from Tiingo.
    Returns a pandas DataFrame.
    """
    end_date = datetime.date.today()
    start_date = end_date - datetime.timedelta(days=730)
    interval_days = 60  # ~60 days = safe for 10-min intervals

    current = start_date
    all_data = []

    print(f"Fetching 10-min data for {ticker} from {start_date} to {end_date}...")

    while current < end_date:
        chunk_end = min(current + datetime.timedelta(days=interval_days), end_date)

        url = f"https://api.tiingo.com/iex/{ticker}/prices"
        headers = {
            'Content-Type': 'application/json'
        }
        params = {
            'startDate': current.isoformat(),
            'endDate': chunk_end.isoformat(),
            'resampleFreq': '10min',
            'columns': 'open,high,low,close,volume',
            'token': api_token
        }

        print(f"  → Fetching from {current} to {chunk_end}...")
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            data = response.json()
            all_data.extend(data)
        else:
            print(f"  ❌ Error {response.status_code}: {response.text}")

        current = chunk_end + datetime.timedelta(days=1)
        time.sleep(1)  # Add delay to avoid hitting rate limits

    # Convert to DataFrame
    df = pd.DataFrame(all_data)
    df["Ticker"] = ticker

    if not df.empty:
        df['date'] = pd.to_datetime(df['date'])
        df.sort_values('date', inplace=True)

    return df


In [23]:
aapl_df["Ticker"] = "AAPL"

In [30]:
aapl_df

,date,open,high,low,close,volume,Ticker
0,2023-04-24 13:30:00+00:00,164.975,165.240,164.415,165.060,60755.0,AAPL
1,2023-04-24 13:40:00+00:00,165.040,165.460,164.940,165.130,14699.0,AAPL
2,2023-04-24 13:50:00+00:00,165.130,165.415,165.075,165.170,6410.0,AAPL
3,2023-04-24 14:00:00+00:00,165.175,165.180,164.820,164.970,11225.0,AAPL
4,2023-04-24 14:10:00+00:00,164.955,165.280,164.820,165.260,15571.0,AAPL
...,...,...,...,...,...,...,...
20316,2025-04-23 15:20:00+00:00,206.350,206.605,204.040,204.625,49798.0,AAPL
20317,2025-04-23 15:30:00+00:00,204.625,204.960,204.180,204.955,43007.0,AAPL
20318,2025-04-23 15:40:00+00:00,204.950,205.430,204.070,204.090,46708.0,AAPL
20319,2025-04-23 15:50:00+00:00,204.090,204.615,203.615,204.265,26392.0,AAPL


In [21]:
# Replace with your Tiingo API key
API_KEY = "619a6ea449a7a5e044a79c9a2293eee49c339034"

# Fetch for Apple
aapl_df = fetch_10min_tiingo_data("AAPL", API_KEY)

# Save to CSV
aapl_df.to_csv("aapl_10min_2yr.csv", index=False)


Fetching 10-min data for AAPL from 2023-04-24 to 2025-04-23...
  → Fetching from 2023-04-24 to 2023-06-23...
  → Fetching from 2023-06-24 to 2023-08-23...
  → Fetching from 2023-08-24 to 2023-10-23...
  → Fetching from 2023-10-24 to 2023-12-23...
  → Fetching from 2023-12-24 to 2024-02-22...
  → Fetching from 2024-02-23 to 2024-04-23...
  → Fetching from 2024-04-24 to 2024-06-23...
  → Fetching from 2024-06-24 to 2024-08-23...
  → Fetching from 2024-08-24 to 2024-10-23...
  → Fetching from 2024-10-24 to 2024-12-23...
  → Fetching from 2024-12-24 to 2025-02-22...
  → Fetching from 2025-02-23 to 2025-04-23...


In [32]:
def insert_PriceHistory(Ticker, data, conn):
    """
    Ticker: str, the name of the table to insert data into.
    data: Dataframe
    """

    data.to_sql(f'{Ticker}PriceHistory', conn, if_exists='append', index=False)
    conn.commit()

In [34]:
insert_PriceHistory("AAPL", aapl_df, conn)

In [35]:
close_connection(conn)